<a href="https://colab.research.google.com/github/bhardwaj1230/NMT/blob/master/get_all_BT_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
from tqdm import tqdm
import os
import pickle
import pandas as pd
import random

destination = ['ALIGNMENT-QUALITY', 'MISALIGNED', 'QUALITY'] 

for l in destination:
    
    location = '/srv/gluster/mtp_shared/corpora/BtB/MegaCorpus_2019-02-14/corpus_renamed/'+str(l)
    loc=[]
    for path, subdirs, files in tqdm(os.walk(location)):
        for name in files:
            loc.append(os.path.join(path, name))

    new_loc=[]
    print('\n # Location : ',l,' : ', len(loc))
    for item in tqdm(loc):
        if item[-2:] == 'en' or item[-2:] == 'fr':
            new_loc.append(item)

    with open('/srv/gluster/users/bhardwajs/en_fr_data/data/complete_flagged/all_loc.en-fr', 'ab') as frp:
        pickle.dump(new_loc,frp)


def read_data(loc):
    objects = []
    with (open(loc, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    return objects

address = read_data("/srv/gluster/users/bhardwajs/en_fr_data/data/complete_flagged/all_loc.en-fr")

address = [item for ad in address for item in ad]
address = set(address)
locations=[]
for item in address:
    locations.append(item[:-2])

locations = list(set(locations))


In [0]:
def get_data(loc, name):
    
    train_en = []
    train_fr = []
    counter = 0
    
    for en_fr in tqdm(loc):

        counter = 1+counter
        
        if os.path.exists(en_fr+str('fr')) and os.path.exists(en_fr+str('en')):

            with open(en_fr+str('fr')) as f:
                frc = [line.strip(' \n') for line in f]
            train_fr.extend(frc)

            with open(en_fr+str('en')) as e:
                eng = [line.strip(' \n') for line in e]
            train_en.extend(eng) 

            if (counter % 1000==0):

                if len(train_fr) == len(train_en):

                    en_data = pd.DataFrame(train_en)
                    fr_data = pd.DataFrame(train_fr)

                    en_data.to_csv('/srv/gluster/users/bhardwajs/en_fr_data/data/complete_flagged/'+str(name)+'.bt-en-fr.en', header=None, index=None, mode='a')
                    fr_data.to_csv('/srv/gluster/users/bhardwajs/en_fr_data/data/complete_flagged/'+str(name)+'.bt-en-fr.fr', header=None, index=None, mode='a')

                    train_en = []
                    train_fr = []
                    
    en_data = pd.DataFrame(train_en)
    fr_data = pd.DataFrame(train_fr)
    print('writing from here now :', len(en_data))
    en_data.to_csv('/srv/gluster/users/bhardwajs/en_fr_data/data/complete_flagged/'+str(name)+'.bt-en-fr.en', header=None, index=None, mode='a')
    fr_data.to_csv('/srv/gluster/users/bhardwajs/en_fr_data/data/complete_flagged/'+str(name)+'.bt-en-fr.fr', header=None, index=None, mode='a')

get_data(locations, 'not_flagged')